# Dataset_Preprocessing

**Content**:
1. [Libraries](#libraries)
2. [Gridworld Creation](#gridworld)
3. [Functions](#funct)
4. [Uniform Dataset Preprocessing](#uniformtransformations)
5. [NonUniform Dataset Preprocessing](#nonuniformtransformations)

## 1. Libraries <a id = "libraries"> 

In [1]:
import numpy as np
import random
import sys
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

## 2. Gridworld Creation <a id = "gridworld">

In [2]:
def randPair(s,e):
    return np.random.randint(s,e), np.random.randint(s,e)

class BoardPiece:

    def __init__(self, name, code, pos):
        self.name = name #name of the piece
        self.code = code #an ASCII character to display on the board
        self.pos = pos #2-tuple e.g. (1,4)

class BoardMask:

    def __init__(self, name, mask, code):
        self.name = name
        self.mask = mask
        self.code = code

    def get_positions(self): #returns tuple of arrays
        return np.nonzero(self.mask)

def zip_positions2d(positions): #positions is tuple of two arrays
    x,y = positions
    return list(zip(x,y))

class GridBoard:

    def __init__(self, size=4):
        self.size = size #Board dimensions, e.g. 4 x 4
        self.components = {} #name : board piece
        self.masks = {}

    def addPiece(self, name, code, pos=(0,0)):
        newPiece = BoardPiece(name, code, pos)
        self.components[name] = newPiece

    #basically a set of boundary elements
    def addMask(self, name, mask, code):
        #mask is a 2D-numpy array with 1s where the boundary elements are
        newMask = BoardMask(name, mask, code)
        self.masks[name] = newMask

    def movePiece(self, name, pos):
        move = True
        for _, mask in self.masks.items():
            if pos in zip_positions2d(mask.get_positions()):
                move = False
        if move:
            self.components[name].pos = pos

    def delPiece(self, name):
        del self.components['name']

    def render(self):
        dtype = '<U2'
        displ_board = np.zeros((self.size, self.size), dtype=dtype)
        displ_board[:] = ' '

        for name, piece in self.components.items():
            displ_board[piece.pos] = piece.code

        for name, mask in self.masks.items():
            displ_board[mask.get_positions()] = mask.code

        return displ_board

    def render_np(self):
        num_pieces = len(self.components) + len(self.masks)
        displ_board = np.zeros((num_pieces, self.size, self.size), dtype=np.uint8)
        layer = 0
        for name, piece in self.components.items():
            pos = (layer,) + piece.pos
            displ_board[pos] = 1
            layer += 1

        for name, mask in self.masks.items():
            x,y = self.masks['boundary'].get_positions()
            z = np.repeat(layer,len(x))
            a = (z,x,y)
            displ_board[a] = 1
            layer += 1
        return displ_board

def addTuple(a,b):
    return tuple([sum(x) for x in zip(a,b)])

In [3]:
class Gridworld:

    def __init__(self, size=4, mode='static'):
        if size >= 4:
            self.board = GridBoard(size=size)
        else:
            print("Minimum board size is 4. Initialized to size 4.")
            self.board = GridBoard(size=4)

        #Add pieces, positions will be updated later
        self.board.addPiece('Player','P',(0,0))
        self.board.addPiece('Goal','+',(0,3))
        self.board.addPiece('Pit','-',(0,3))
        self.board.addPiece('Wall','W',(2,3))

        if mode == 'static':
            self.initGridStatic()
        elif mode == 'player':
            self.initGridPlayer()
        else:
            self.initGridRand()

    #Initialize stationary grid, all items are placed deterministically
    def initGridStatic(self):
        #Setup static pieces
        self.board.components['Player'].pos = randPair(0,self.board.size) #Row, Column
        self.board.components['Goal'].pos = (3,0)
        self.board.components['Pit'].pos = (3,1)
        self.board.components['Wall'].pos = (3,2)
        
        if (not self.validateBoard()):
            #print('Invalid grid. Rebuilding..')
            self.initGridStatic()


    #Check if board is initialized appropriately (no overlapping pieces)
    #also remove impossible-to-win boards
    def validateBoard(self):
        valid = True

        player = self.board.components['Player']
        goal = self.board.components['Goal']
        wall = self.board.components['Wall']
        pit = self.board.components['Pit']

        all_positions = [piece for name,piece in self.board.components.items()]
        all_positions = [player.pos, goal.pos, wall.pos, pit.pos]
        if len(all_positions) > len(set(all_positions)):
            return False

        corners = [(0,0),(0,self.board.size), (self.board.size,0), (self.board.size,self.board.size)]
        #if player is in corner, can it move? if goal is in corner, is it blocked?
        if player.pos in corners or goal.pos in corners:
            val_move_pl = [self.validateMove('Player', addpos) for addpos in [(0,1),(1,0),(-1,0),(0,-1)]]
            val_move_go = [self.validateMove('Goal', addpos) for addpos in [(0,1),(1,0),(-1,0),(0,-1)]]
            if 0 not in val_move_pl or 0 not in val_move_go:
                #print(self.display())
                #print("Invalid board. Re-initializing...")
                valid = False

        return valid

    #Initialize player in random location, but keep wall, goal and pit stationary
    def initGridPlayer(self):
        #height x width x depth (number of pieces)
        self.initGridStatic()
        #place player
        self.board.components['Player'].pos = randPair(0,self.board.size)

        if (not self.validateBoard()):
            #print('Invalid grid. Rebuilding..')
            self.initGridPlayer()

    #Initialize grid so that goal, pit, wall, player are all randomly placed
    def initGridRand(self):
        #height x width x depth (number of pieces)
        self.board.components['Player'].pos = randPair(0,self.board.size)
        self.board.components['Goal'].pos = randPair(0,self.board.size)
        self.board.components['Pit'].pos = randPair(0,self.board.size)
        self.board.components['Wall'].pos = randPair(0,self.board.size)

        if (not self.validateBoard()):
            #print('Invalid grid. Rebuilding..')
            self.initGridRand()

    def validateMove(self, piece, addpos=(0,0)):
        outcome = 0 #0 is valid, 1 invalid, 2 lost game
        pit = self.board.components['Pit'].pos
        wall = self.board.components['Wall'].pos
        new_pos = addTuple(self.board.components[piece].pos, addpos)
        if new_pos == wall:
            outcome = 1 #block move, player can't move to wall
        elif max(new_pos) > (self.board.size-1):    #if outside bounds of board
            outcome = 1
        elif min(new_pos) < 0: #if outside bounds
            outcome = 1
        elif new_pos == pit:
            outcome = 2

        return outcome

    def makeMove(self, action):
        #need to determine what object (if any) is in the new grid spot the player is moving to
        #actions in {u,d,l,r}
        def checkMove(addpos):
            if self.validateMove('Player', addpos) in [0,2]:
                new_pos = addTuple(self.board.components['Player'].pos, addpos)
                self.board.movePiece('Player', new_pos)

        if action == 'u': #up
            checkMove((-1,0))
        elif action == 'd': #down
            checkMove((1,0))
        elif action == 'l': #left
            checkMove((0,-1))
        elif action == 'r': #right
            checkMove((0,1))
        else:
            pass

    def reward(self):
        if (self.board.components['Player'].pos == self.board.components['Pit'].pos):
            return -10
        elif (self.board.components['Player'].pos == self.board.components['Goal'].pos):
            return 10
        else:
            return -1

    def display(self):
        return self.board.render()

In [4]:
action_set = {
    0: 'u',
    1: 'd',
    2: 'l',
    3: 'r',
}

## 3. Functions <a id = 'funct'> 

In [5]:
"""
from2dto1d:
    Args: 
        pos(tuple):2d position of the objects(Player, Wall, Goal, Pit) in the gridworld
    Return:
        return(integer): 1d position of the objects(Player, Wall, Goal, Pit)in the gridworld
""" 
def from2dto1d(pos):
    if pos == '(0, 0)':
        return 0
    if pos == '(0, 1)':
        return 1
    if pos == '(0, 2)':
        return 2
    if pos == '(0, 3)':
        return 3
    if pos == '(1, 0)':
        return 4
    if pos == '(1, 1)':
        return 5
    if pos == '(1, 2)':
        return 6
    if pos == '(1, 3)':
        return 7
    if pos == '(2, 0)':
        return 8
    if pos == '(2, 1)':
        return 9
    if pos == '(2, 2)':
        return 10
    if pos == '(2, 3)':
        return 11
    if pos == '(3, 0)':
        return 12
    if pos == '(3, 1)':
        return 13
    if pos == '(3, 2)':
        return 14
    if pos == '(3, 3)':
        return 15

In [6]:
def from1dto2d(pos):
    if pos == 0:
        return (0, 0)
    if pos == 1:
        return (0, 1)
    if pos == 2:
        return (0, 2)
    if pos == 3:
        return (0, 3)
    if pos == 4:
        return (1, 0)
    if pos == 5:
        return (1, 1)
    if pos == 6:
        return (1, 2)
    if pos == 7:
        return (1, 3)
    if pos == 8:
        return (2, 0)
    if pos == 9:
        return (2, 1)
    if pos == 10:
        return (2, 2)
    if pos == 11:
        return (2, 3)
    if pos == 12:
        return (3, 0)
    if pos == 13:
        return (3, 1)
    if pos == 14:
        return (3, 2)
    if pos == 15:
        return (3, 3)

## 4. Uniform Dataset Preprocessing <a id = "uniformtransformations">

In [7]:
df = pd.read_csv("DatasetBeforeUniform_GYM.csv")
df.drop('Unnamed: 0', axis=1, inplace=True)   
df

,Current_State,Pit_Position,Goal_Position,Wall_Position,Current_Reward,Q1_UP,Q2_DOWN,Q3_LEFT,Q4_RIGHT
0,0,0,2,1,-10,4.946462,4.695930,4.974383,4.760919
1,2,0,2,1,-1,8.128312,8.142357,7.115776,7.350907
2,3,0,2,1,10,6.755848,5.773548,9.332119,7.258504
3,4,0,2,1,-1,3.076944,2.692700,4.710314,6.191459
4,5,0,2,1,-1,6.541164,4.304107,5.105873,8.097490
...,...,...,...,...,...,...,...,...,...
50395,10,15,13,14,-1,5.938396,5.431857,7.606215,4.054748
50396,11,15,13,14,-1,2.553046,2.540612,5.090322,4.031656
50397,12,15,13,14,10,4.916274,4.017373,3.690299,6.726671
50398,13,15,13,14,-1,8.272136,6.379996,4.600275,1.773542


### Rearrangement of dataset

In [8]:
counter = 0

h = 0
w, h = 6, 3000000
matrix = [[0 for x in range(w)] for y in range(h)] 

for i in range (0, df.shape[0]):
    game = Gridworld(size=4, mode='random')
    game.board.components['Player'].pos = from1dto2d(df["Current_State"][i])
    game.board.components['Goal'].pos = from1dto2d(df["Goal_Position"][i])
    game.board.components['Wall'].pos = from1dto2d(df["Wall_Position"][i])
    game.board.components['Pit'].pos = from1dto2d(df["Pit_Position"][i])
    
    pi = from2dto1d((str(game.board.components['Player'].pos)))
    game.makeMove('u')
    p = from2dto1d((str(game.board.components['Player'].pos)))
    if p == pi:
        pass
    else:
        matrix[counter][0] = from2dto1d((str(game.board.components['Player'].pos)))
        matrix[counter][1] = from2dto1d((str(game.board.components['Goal'].pos)))
        matrix[counter][2] = from2dto1d((str(game.board.components['Wall'].pos)))
        matrix[counter][3] = from2dto1d((str(game.board.components['Pit'].pos)))
        matrix[counter][4] = df["Q1_UP"][i]
        reward = game.reward()
        matrix[counter][5] = reward
        game.makeMove('d')
        counter += 1 
        
    pi = from2dto1d((str(game.board.components['Player'].pos)))    
    game.makeMove('d')
    p = from2dto1d((str(game.board.components['Player'].pos)))
    if p == pi:
        pass
    else:
        matrix[counter][0] = from2dto1d((str(game.board.components['Player'].pos)))
        matrix[counter][1] = from2dto1d((str(game.board.components['Goal'].pos)))
        matrix[counter][2] = from2dto1d((str(game.board.components['Wall'].pos)))
        matrix[counter][3] = from2dto1d((str(game.board.components['Pit'].pos)))
        matrix[counter][4] = df["Q2_DOWN"][i]
        reward = game.reward()
        matrix[counter][5] = reward
        game.makeMove('u')
        counter += 1 
        
    pi = from2dto1d((str(game.board.components['Player'].pos)))    
    game.makeMove('l')
    p = from2dto1d((str(game.board.components['Player'].pos)))
    if p == pi:
        pass
    else:
        matrix[counter][0] = from2dto1d((str(game.board.components['Player'].pos)))
        matrix[counter][1] = from2dto1d((str(game.board.components['Goal'].pos)))
        matrix[counter][2] = from2dto1d((str(game.board.components['Wall'].pos)))
        matrix[counter][3] = from2dto1d((str(game.board.components['Pit'].pos)))
        matrix[counter][4] = df["Q3_LEFT"][i]
        reward = game.reward()
        matrix[counter][5] = reward
        game.makeMove('r')
        counter += 1 
        
    pi = from2dto1d((str(game.board.components['Player'].pos)))   
    
    game.makeMove('r')
    p = from2dto1d((str(game.board.components['Player'].pos)))
    if p == pi:
        pass
    else:
        matrix[counter][0] = from2dto1d((str(game.board.components['Player'].pos)))
        matrix[counter][1] = from2dto1d((str(game.board.components['Goal'].pos)))
        matrix[counter][2] = from2dto1d((str(game.board.components['Wall'].pos)))
        matrix[counter][3] = from2dto1d((str(game.board.components['Pit'].pos)))
        matrix[counter][4] = df["Q4_RIGHT"][i]
        reward = game.reward()
        matrix[counter][5] = reward
        game.makeMove('l')
        counter += 1 

In [9]:
column_names = ["Player",  "Goal", "Wall", "Pit", "Q_value", "Reward"]
df_matrix = pd.DataFrame(matrix, columns = column_names)
df_matrix = df_matrix.loc[(df_matrix != 0).any(axis=1)]
df_matrix

,Player,Goal,Wall,Pit,Q_value,Reward
0,4,2,1,0,4.695930,-1
1,6,2,1,0,8.142357,-1
2,3,2,1,0,7.350907,-1
3,7,2,1,0,5.773548,-1
4,2,2,1,0,9.332119,10
...,...,...,...,...,...,...
141115,8,13,14,15,4.916274,-1
141116,13,13,14,15,6.726671,10
141117,9,13,14,15,8.272136,-1
141118,12,13,14,15,4.600275,-1


In [10]:
# Drop duplicate positions
df_new = df_matrix.drop_duplicates(keep='last',subset=[ 'Player', 'Goal', 'Wall', 'Pit'])
df_new = df_new.reset_index()
df_new.drop('index', axis=1, inplace=True)   
df_new

,Player,Goal,Wall,Pit,Q_value,Reward
0,0,2,1,0,3.076944,-10
1,2,2,1,0,9.783095,10
2,3,2,1,0,8.355206,-1
3,4,2,1,0,3.889865,-1
4,5,2,1,0,6.721507,-1
...,...,...,...,...,...,...
50395,8,13,14,15,4.916274,-1
50396,13,13,14,15,6.726671,10
50397,9,13,14,15,8.272136,-1
50398,12,13,14,15,4.600275,-1


### Bellman Operator Dataset Creation

In [11]:
no_dup_df = df_new
no_dup_df

,Player,Goal,Wall,Pit,Q_value,Reward
0,0,2,1,0,3.076944,-10
1,2,2,1,0,9.783095,10
2,3,2,1,0,8.355206,-1
3,4,2,1,0,3.889865,-1
4,5,2,1,0,6.721507,-1
...,...,...,...,...,...,...
50395,8,13,14,15,4.916274,-1
50396,13,13,14,15,6.726671,10
50397,9,13,14,15,8.272136,-1
50398,12,13,14,15,4.600275,-1


In [12]:
counter = 0

h = 0
w, h = 7, 300000
matrix = [[0 for x in range(w)] for y in range(h)] 

for i in range (0, no_dup_df.shape[0]):
    game = Gridworld(size=4, mode='random')
    game.board.components['Player'].pos = from1dto2d(no_dup_df["Player"][i])
    game.board.components['Goal'].pos = from1dto2d(no_dup_df["Goal"][i])
    game.board.components['Wall'].pos = from1dto2d(no_dup_df["Wall"][i])
    game.board.components['Pit'].pos = from1dto2d(no_dup_df["Pit"][i])
    
    q_value = []
    reward_arr = []
    
    pi = from2dto1d((str(game.board.components['Player'].pos)))
    
    goal = int(no_dup_df["Goal"][i])
    pit = int(no_dup_df["Pit"][i])
    wall = int(no_dup_df["Wall"][i])
    
    matrix[counter][0] = pi
    matrix[counter][1] = no_dup_df["Goal"][i]
    matrix[counter][2] = no_dup_df["Wall"][i]
    matrix[counter][3] = no_dup_df["Pit"][i]
    matrix[counter][4] = no_dup_df["Q_value"][i]
    matrix[counter][5] = no_dup_df["Reward"][i]
    
    game.makeMove('u')
    p = from2dto1d((str(game.board.components['Player'].pos)))

    if p == pi:
        q_value.append(-10)
        reward_arr.append(-100)
    else:
        q_value.append(no_dup_df[(no_dup_df["Player"] == int(p)) & (no_dup_df["Goal"] == goal)  & (no_dup_df["Wall"] == wall) & (no_dup_df["Pit"] == pit)]['Q_value'].values[0])
        reward = game.reward()
        reward_arr.append(reward)
        game.makeMove('d')
        
        
    pi = from2dto1d((str(game.board.components['Player'].pos)))    
    game.makeMove('d')
    p = from2dto1d((str(game.board.components['Player'].pos)))
            
    if p == pi:
        q_value.append(-10)
        reward_arr.append(-100)
    else:
        q_value.append(no_dup_df[(no_dup_df["Player"] == p) & (no_dup_df["Goal"] == goal)  & (no_dup_df["Wall"] == wall) & (no_dup_df["Pit"] == pit)]['Q_value'].values[0])
        reward = game.reward()
        reward_arr.append(reward)
        game.makeMove('u')
        
        
    pi = from2dto1d((str(game.board.components['Player'].pos)))    
    game.makeMove('l')
    p = from2dto1d((str(game.board.components['Player'].pos)))
            
    if p == pi:
        q_value.append(-10)
        reward_arr.append(-100)
    else:
        q_value.append(no_dup_df[(no_dup_df["Player"] == p) & (no_dup_df["Goal"] == goal)  & (no_dup_df["Wall"] == wall) & (no_dup_df["Pit"] == pit)]['Q_value'].values[0])
        reward = game.reward()
        reward_arr.append(reward)
        game.makeMove('r')
        
        
    pi = from2dto1d((str(game.board.components['Player'].pos)))       
    game.makeMove('r')
    p = from2dto1d((str(game.board.components['Player'].pos)))
            
    if p == pi:
        q_value.append(-10)
        reward_arr.append(-100)
    else:
        q_value.append(no_dup_df[(no_dup_df["Player"] == p) & (no_dup_df["Goal"] == goal)  & (no_dup_df["Wall"] == wall) & (no_dup_df["Pit"] == pit)]['Q_value'].values[0])
        reward = game.reward()
        reward_arr.append(reward)
        game.makeMove('l')
        
    
    max_qvalue = q_value[np.argmax(q_value)]
    reward_max = reward_arr[np.argmax(q_value)]
    bellman = max_qvalue*0.9 + reward_max
    if pi == goal or bellman > 10:
        matrix[counter][6] = 10
    else:
        matrix[counter][6] = bellman
    counter += 1

In [13]:
import pandas as pd
column_names = ["Player","Goal", "Wall", "Pit", "Q_Current", "Reward_Current", "Bellman_Operator"
                ]
bell_df = pd.DataFrame(matrix, columns = column_names)
bell_df = bell_df.loc[(bell_df != 0).any(axis=1)]
bell_df

,Player,Goal,Wall,Pit,Q_Current,Reward_Current,Bellman_Operator
0,0,2,1,0,3.076944,-10,2.500878
1,2,2,1,0,9.783095,10,10.000000
2,3,2,1,0,8.355206,-1,10.000000
3,4,2,1,0,3.889865,-1,5.049356
4,5,2,1,0,6.721507,-1,5.925810
...,...,...,...,...,...,...,...
50395,8,13,14,15,4.916274,-1,6.444922
50396,13,13,14,15,6.726671,10,10.000000
50397,9,13,14,15,8.272136,-1,5.490914
50398,12,13,14,15,4.600275,-1,10.000000


In [14]:
bell_df.to_csv('DatasetAfterUniform_GYM.csv')

## 5. NonUniform Dataset Preprocessing <a id = "nonuniformtransformations">

In [15]:
df = pd.read_csv("DatasetBeforeNonUniform_GYM.csv")
df.drop('Unnamed: 0', axis=1, inplace=True)   
df

,Epochs,Player,Pit,Goal,Wall,Action,Reward,Next_State,Epsilon_Boolean,Q_MAX,Q1_UP,Q2_DOWN,Q3_LEFT,Q4_RIGHT,Q1_UP_NEXT,Q2_DOWN_NEXT,Q3_LEFT_NEXT,Q4_RIGHT_NEXT,Q_MAX_NEXT,Y
0,5000,7,14,4,13,2,-1,6,0,6.062968,3.479241,5.422428,6.062968,3.602073,4.272051,6.197496,7.599072,4.462092,7.599072,5.839165
1,5000,6,14,4,13,2,-1,5,0,7.599072,4.272051,6.197496,7.599072,4.462092,6.417338,7.986413,11.033390,5.889728,11.033390,8.930051
2,5000,5,14,4,13,2,10,4,0,11.033390,6.417338,7.986413,11.033390,5.889728,6.913105,6.788280,7.669920,5.922499,7.669920,10.000000
3,5001,15,10,6,2,0,-1,11,0,5.646697,5.646697,4.463362,2.770998,4.443552,7.753917,5.452022,3.113808,5.731588,7.753917,5.978525
4,5001,11,10,6,2,0,-1,7,0,7.753917,7.753917,5.452022,3.113808,5.731588,7.503795,3.967476,8.873061,7.824474,8.873061,6.985755
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18576,9998,10,13,5,11,0,-1,6,0,8.031532,8.031532,3.936372,6.951592,3.961432,8.356394,7.289367,10.148597,6.493959,10.148597,8.133737
18577,9998,6,13,5,11,2,10,5,0,10.162380,8.402393,7.285714,10.162380,6.481378,9.240396,8.338165,7.988938,8.698484,9.240396,10.000000
18578,9999,15,7,12,1,2,-1,14,0,6.003728,3.690572,4.656336,6.003728,4.239810,6.086253,5.985104,7.620012,6.476632,7.620012,5.858011
18579,9999,14,7,12,1,2,-1,13,0,7.623302,6.092485,5.976284,7.623302,6.468684,8.303739,8.453712,9.373241,7.784957,9.373241,7.435917


In [16]:
# Drop duplicate positions
df_new = df.drop_duplicates(keep='last',subset=['Player', 'Goal', 'Wall', 'Pit'])
df_new = df_new[df_new["Epsilon_Boolean"] == 0]
df_new = df_new.reset_index()
df_new.drop('index', axis=1, inplace=True) 
df_new

,Epochs,Player,Pit,Goal,Wall,Action,Reward,Next_State,Epsilon_Boolean,Q_MAX,Q1_UP,Q2_DOWN,Q3_LEFT,Q4_RIGHT,Q1_UP_NEXT,Q2_DOWN_NEXT,Q3_LEFT_NEXT,Q4_RIGHT_NEXT,Q_MAX_NEXT,Y
0,5000,7,14,4,13,2,-1,6,0,6.062968,3.479241,5.422428,6.062968,3.602073,4.272051,6.197496,7.599072,4.462092,7.599072,5.839165
1,5000,6,14,4,13,2,-1,5,0,7.599072,4.272051,6.197496,7.599072,4.462092,6.417338,7.986413,11.033390,5.889728,11.033390,8.930051
2,5000,5,14,4,13,2,10,4,0,11.033390,6.417338,7.986413,11.033390,5.889728,6.913105,6.788280,7.669920,5.922499,7.669920,10.000000
3,5001,15,10,6,2,0,-1,11,0,5.646697,5.646697,4.463362,2.770998,4.443552,7.753917,5.452022,3.113808,5.731588,7.753917,5.978525
4,5001,11,10,6,2,0,-1,7,0,7.753917,7.753917,5.452022,3.113808,5.731588,7.503795,3.967476,8.873061,7.824474,8.873061,6.985755
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10894,9998,10,13,5,11,0,-1,6,0,8.031532,8.031532,3.936372,6.951592,3.961432,8.356394,7.289367,10.148597,6.493959,10.148597,8.133737
10895,9998,6,13,5,11,2,10,5,0,10.162380,8.402393,7.285714,10.162380,6.481378,9.240396,8.338165,7.988938,8.698484,9.240396,10.000000
10896,9999,15,7,12,1,2,-1,14,0,6.003728,3.690572,4.656336,6.003728,4.239810,6.086253,5.985104,7.620012,6.476632,7.620012,5.858011
10897,9999,14,7,12,1,2,-1,13,0,7.623302,6.092485,5.976284,7.623302,6.468684,8.303739,8.453712,9.373241,7.784957,9.373241,7.435917


In [17]:
counter = 0

h = 0
w, h = 6, 3000000
matrix = [[0 for x in range(w)] for y in range(h)] 

for i in range (0, df_new.shape[0]):
    game = Gridworld(size=4, mode='random')
    game.board.components['Player'].pos = from1dto2d(df_new["Player"][i])
    game.board.components['Goal'].pos = from1dto2d(df_new["Goal"][i])
    game.board.components['Wall'].pos = from1dto2d(df_new["Wall"][i])
    game.board.components['Pit'].pos = from1dto2d(df_new["Pit"][i])
    
    pi = from2dto1d((str(game.board.components['Player'].pos)))
    game.makeMove('u')
    p = from2dto1d((str(game.board.components['Player'].pos)))
    if p == pi:
        pass
    else:
        matrix[counter][0] = from2dto1d((str(game.board.components['Player'].pos)))
        matrix[counter][1] = from2dto1d((str(game.board.components['Goal'].pos)))
        matrix[counter][2] = from2dto1d((str(game.board.components['Wall'].pos)))
        matrix[counter][3] = from2dto1d((str(game.board.components['Pit'].pos)))
        matrix[counter][4] = df_new["Q1_UP"][i]
        reward = game.reward()
        matrix[counter][5] = reward
        game.makeMove('d')
        counter += 1 
        
    pi = from2dto1d((str(game.board.components['Player'].pos)))    
    game.makeMove('d')
    p = from2dto1d((str(game.board.components['Player'].pos)))
    if p == pi:
        pass
    else:
        matrix[counter][0] = from2dto1d((str(game.board.components['Player'].pos)))
        matrix[counter][1] = from2dto1d((str(game.board.components['Goal'].pos)))
        matrix[counter][2] = from2dto1d((str(game.board.components['Wall'].pos)))
        matrix[counter][3] = from2dto1d((str(game.board.components['Pit'].pos)))
        matrix[counter][4] = df_new["Q2_DOWN"][i]
        reward = game.reward()
        matrix[counter][5] = reward
        game.makeMove('u')
        counter += 1 
        
    pi = from2dto1d((str(game.board.components['Player'].pos)))    
    game.makeMove('l')
    p = from2dto1d((str(game.board.components['Player'].pos)))
    if p == pi:
        pass
    else:
        matrix[counter][0] = from2dto1d((str(game.board.components['Player'].pos)))
        matrix[counter][1] = from2dto1d((str(game.board.components['Goal'].pos)))
        matrix[counter][2] = from2dto1d((str(game.board.components['Wall'].pos)))
        matrix[counter][3] = from2dto1d((str(game.board.components['Pit'].pos)))
        matrix[counter][4] = df_new["Q3_LEFT"][i]
        reward = game.reward()
        matrix[counter][5] = reward
        game.makeMove('r')
        counter += 1 
        
    pi = from2dto1d((str(game.board.components['Player'].pos)))   
    
    game.makeMove('r')
    p = from2dto1d((str(game.board.components['Player'].pos)))
    if p == pi:
        pass
    else:
        matrix[counter][0] = from2dto1d((str(game.board.components['Player'].pos)))
        matrix[counter][1] = from2dto1d((str(game.board.components['Goal'].pos)))
        matrix[counter][2] = from2dto1d((str(game.board.components['Wall'].pos)))
        matrix[counter][3] = from2dto1d((str(game.board.components['Pit'].pos)))
        matrix[counter][4] = df_new["Q4_RIGHT"][i]
        reward = game.reward()
        matrix[counter][5] = reward
        game.makeMove('l')
        counter += 1 

In [18]:
column_names = ["Player",  "Goal", "Wall", "Pit", "Q_value", "Reward"]
df_matrix = pd.DataFrame(matrix, columns = column_names)
df_matrix = df_matrix.loc[(df_matrix != 0).any(axis=1)]
df_matrix

,Player,Goal,Wall,Pit,Q_value,Reward
0,3,4,13,14,3.479241,-1
1,11,4,13,14,5.422428,-1
2,6,4,13,14,6.062968,-1
3,2,4,13,14,4.272051,-1
4,10,4,13,14,6.197496,-1
...,...,...,...,...,...,...
33332,13,12,1,7,7.623302,-1
33333,15,12,1,7,6.468684,-1
33334,9,12,1,7,8.301615,-1
33335,12,12,1,7,9.378736,10


In [19]:
# Drop duplicate positions
df_final = df_matrix.drop_duplicates(keep='last',subset=[ 'Player', 'Goal', 'Wall', 'Pit'])
df_final = df_final.reset_index()
df_final.drop('index', axis=1, inplace=True)   
df_final

,Player,Goal,Wall,Pit,Q_value,Reward
0,3,4,13,14,3.479241,-1
1,2,4,13,14,4.272051,-1
2,1,4,13,14,6.417338,-1
3,7,6,2,10,7.753917,-1
4,3,6,2,10,7.503795,-1
...,...,...,...,...,...,...
23844,13,12,1,7,7.623302,-1
23845,15,12,1,7,6.468684,-1
23846,9,12,1,7,8.301615,-1
23847,12,12,1,7,9.378736,10


In [20]:
df_final.Reward.value_counts()

-1     19913
 10     2544
-10     1392
Name: Reward, dtype: int64

In [21]:
df_new = df_new[['Player', 'Goal', 'Wall', 'Pit', 'Q1_UP' ,'Q2_DOWN', 'Q3_LEFT' , 'Q4_RIGHT']].copy()
df_new

,Player,Goal,Wall,Pit,Q1_UP,Q2_DOWN,Q3_LEFT,Q4_RIGHT
0,7,4,13,14,3.479241,5.422428,6.062968,3.602073
1,6,4,13,14,4.272051,6.197496,7.599072,4.462092
2,5,4,13,14,6.417338,7.986413,11.033390,5.889728
3,15,6,2,10,5.646697,4.463362,2.770998,4.443552
4,11,6,2,10,7.753917,5.452022,3.113808,5.731588
...,...,...,...,...,...,...,...,...
10894,10,5,11,13,8.031532,3.936372,6.951592,3.961432
10895,6,5,11,13,8.402393,7.285714,10.162380,6.481378
10896,15,12,1,7,3.690572,4.656336,6.003728,4.239810
10897,14,12,1,7,6.092485,5.976284,7.623302,6.468684


In [22]:
joined_df = pd.merge(df_new, df_final[['Player', 'Goal', 'Wall', 'Pit', 'Q_value', 'Reward']], on=['Player', 'Goal', 'Wall', 'Pit'], how='left')
joined_df = joined_df.dropna()
joined_df = joined_df.reset_index()
joined_df.drop('index', axis=1, inplace=True)   
joined_df

,Player,Goal,Wall,Pit,Q1_UP,Q2_DOWN,Q3_LEFT,Q4_RIGHT,Q_value,Reward
0,7,4,13,14,3.479241,5.422428,6.062968,3.602073,4.311518,-1.0
1,6,4,13,14,4.272051,6.197496,7.599072,4.462092,6.047539,-1.0
2,5,4,13,14,6.417338,7.986413,11.033390,5.889728,8.547612,-1.0
3,15,6,2,10,5.646697,4.463362,2.770998,4.443552,2.999327,-1.0
4,11,6,2,10,7.753917,5.452022,3.113808,5.731588,3.967476,-1.0
...,...,...,...,...,...,...,...,...,...,...
10314,10,5,11,13,8.031532,3.936372,6.951592,3.961432,7.285714,-1.0
10315,6,5,11,13,8.402393,7.285714,10.162380,6.481378,8.031532,-1.0
10316,15,12,1,7,3.690572,4.656336,6.003728,4.239810,6.468684,-1.0
10317,14,12,1,7,6.092485,5.976284,7.623302,6.468684,7.789087,-1.0


### Bellman Operator Dataset Collection

In [23]:
no_dup_df = joined_df
no_dup_df

,Player,Goal,Wall,Pit,Q1_UP,Q2_DOWN,Q3_LEFT,Q4_RIGHT,Q_value,Reward
0,7,4,13,14,3.479241,5.422428,6.062968,3.602073,4.311518,-1.0
1,6,4,13,14,4.272051,6.197496,7.599072,4.462092,6.047539,-1.0
2,5,4,13,14,6.417338,7.986413,11.033390,5.889728,8.547612,-1.0
3,15,6,2,10,5.646697,4.463362,2.770998,4.443552,2.999327,-1.0
4,11,6,2,10,7.753917,5.452022,3.113808,5.731588,3.967476,-1.0
...,...,...,...,...,...,...,...,...,...,...
10314,10,5,11,13,8.031532,3.936372,6.951592,3.961432,7.285714,-1.0
10315,6,5,11,13,8.402393,7.285714,10.162380,6.481378,8.031532,-1.0
10316,15,12,1,7,3.690572,4.656336,6.003728,4.239810,6.468684,-1.0
10317,14,12,1,7,6.092485,5.976284,7.623302,6.468684,7.789087,-1.0


In [24]:
counter = 0

h = 0
w, h = 7, 300000
matrix = [[0 for x in range(w)] for y in range(h)] 

for i in range (0, no_dup_df.shape[0]):
    game = Gridworld(size=4, mode='random')
    game.board.components['Player'].pos = from1dto2d(no_dup_df["Player"][i])
    game.board.components['Goal'].pos = from1dto2d(no_dup_df["Goal"][i])
    game.board.components['Wall'].pos = from1dto2d(no_dup_df["Wall"][i])
    game.board.components['Pit'].pos = from1dto2d(no_dup_df["Pit"][i])
    
    q_value = []
    reward_arr = []
    
    pi = from2dto1d((str(game.board.components['Player'].pos)))
    
    goal = int(no_dup_df["Goal"][i])
    pit = int(no_dup_df["Pit"][i])
    wall = int(no_dup_df["Wall"][i])
    
    matrix[counter][0] = pi
    matrix[counter][1] = no_dup_df["Goal"][i]
    matrix[counter][2] = no_dup_df["Wall"][i]
    matrix[counter][3] = no_dup_df["Pit"][i]
    matrix[counter][4] = no_dup_df["Q_value"][i]
    matrix[counter][5] = no_dup_df["Reward"][i]
    
    q_value.append(no_dup_df["Q1_UP"][i])
    q_value.append(no_dup_df["Q2_DOWN"][i])
    q_value.append(no_dup_df["Q3_LEFT"][i])
    q_value.append(no_dup_df["Q4_RIGHT"][i])
    
    game.makeMove('u')
    p = from2dto1d((str(game.board.components['Player'].pos)))

    if p == pi:
        reward_arr.append(-100)
    else:
        reward = game.reward()
        reward_arr.append(reward)
        game.makeMove('d')
        
        
    pi = from2dto1d((str(game.board.components['Player'].pos)))    
    game.makeMove('d')
    p = from2dto1d((str(game.board.components['Player'].pos)))
            
    if p == pi:
        reward_arr.append(-100)
    else:
        reward = game.reward()
        reward_arr.append(reward)
        game.makeMove('u')
        
        
    pi = from2dto1d((str(game.board.components['Player'].pos)))    
    game.makeMove('l')
    p = from2dto1d((str(game.board.components['Player'].pos)))
            
    if p == pi:
        reward_arr.append(-100)
    else:
        reward = game.reward()
        reward_arr.append(reward)
        game.makeMove('r')
        
        
    pi = from2dto1d((str(game.board.components['Player'].pos)))       
    game.makeMove('r')
    p = from2dto1d((str(game.board.components['Player'].pos)))
            
    if p == pi:
        reward_arr.append(-100)
    else:
        reward = game.reward()
        reward_arr.append(reward)
        game.makeMove('l')
        
    
    max_qvalue = q_value[np.argmax(q_value)]
    reward_max = reward_arr[np.argmax(q_value)]
    bellman = max_qvalue*0.9 + reward_max
    if pi == goal or bellman > 10:
        matrix[counter][6] = 10
    else:
        matrix[counter][6] = bellman
    counter += 1

In [25]:
import pandas as pd
column_names = ["Player","Goal", "Wall", "Pit", "Q_Current", "Reward_Current", "Bellman_Operator"
                ]
bell_df = pd.DataFrame(matrix, columns = column_names)
bell_df = bell_df.loc[(bell_df != 0).any(axis=1)]
bell_df

,Player,Goal,Wall,Pit,Q_Current,Reward_Current,Bellman_Operator
0,7,4,13,14,4.311518,-1.0,4.456671
1,6,4,13,14,6.047539,-1.0,5.839165
2,5,4,13,14,8.547612,-1.0,10.000000
3,15,6,2,10,2.999327,-1.0,4.082027
4,11,6,2,10,3.967476,-1.0,5.978525
...,...,...,...,...,...,...,...
10314,10,5,11,13,7.285714,-1.0,6.228379
10315,6,5,11,13,8.031532,-1.0,10.000000
10316,15,12,1,7,6.468684,-1.0,4.403355
10317,14,12,1,7,7.789087,-1.0,5.860971


In [26]:
bell_df.to_csv('DatasetAfterNonUniform_GYM.csv')